In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
import sys
sys.path.append('../../')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *

# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 3, 'Channels for input')
flags.DEFINE_integer('classics', 51, 'The num of class')
FLAGS = flags.FLAGS

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
outfile = './rgb_scratch_data_split1/test64/'


def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    pre_model_save_dir = "./models/HMDB_scratch_fix/rgb_scratch_10000_6_64_0.0001_decay_low_split1"
    test_list_file = '../../list/hmdb_list/testlist1.list'
    file = list(open(test_list_file, 'r'))
    num_test_videos = len(file)
    print("Number of test videos={}".format(num_test_videos))
    with tf.Graph().as_default():
        rgb_images_placeholder, _, labels_placeholder, is_training = placeholder_inputs(
                        FLAGS.batch_size * gpu_num,
                        FLAGS.num_frame_per_clib,
                        FLAGS.crop_size,
                        FLAGS.rgb_channels
                        )

        with tf.variable_scope('RGB'):
            logit, _ = InceptionI3d(
                                num_classes=FLAGS.classics,
                                spatial_squeeze=True,
                                final_endpoint='Mixed_5c',
                                name='inception_i3d'
                                )(rgb_images_placeholder, is_training)
        norm_score = tf.nn.softmax(logit)

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        # Create a session for running Ops on the Graph.
        sess = tf.Session(
                        config=tf.ConfigProto(allow_soft_placement=True)
                        )
        sess.run(init)

    ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        print ("load complete!")

    all_steps = num_test_videos
    top1_list = []
    for step in xrange(all_steps):
        start_time = time.time()
        s_index = 0
        predicts = []
        data = {}
        top1 = False
        data_new = np.array([])
        while True:
            val_images, _, val_labels, s_index, is_end = input_test.read_clip_and_label(
                            filename=file[step],
                            batch_size=FLAGS.batch_size * gpu_num,
                            s_index=s_index,
                            num_frames_per_clip=FLAGS.num_frame_per_clib,
                            crop_size=FLAGS.crop_size,
                            )

            data_features = sess.run(logit,
                               feed_dict={
                                            rgb_images_placeholder: val_images,
                                            labels_placeholder: val_labels,
                                            is_training: False
                                            })

            temp = np.mean(data_features, axis=2)
            tmp = np.mean(temp, axis=2)
            tmp = np.squeeze(tmp)

            if data_new.shape[0] == 0:
                data_new = tmp
            else:
                data_new = np.concatenate([data_new,tmp])

            labell = np.zeros((1,51))
            labell[0,int(val_labels)] = 1

            #print(labell)

            if is_end:
                ndata = int(data_new.shape[0])
                nn = 64
                if ndata <=nn:
                    data_new = np.concatenate([data_new,data_new],  axis=0)
                    data_new = np.concatenate([data_new,data_new],  axis=0)
                    ndata = int(data_new.shape[0])
                
                sampling = ndata/nn
                print(sampling)
                print(data_new.shape)
                ndata2 = data_new[0:nn*sampling,:]
                ndata3 = ndata2[0::sampling,:]

                data['feat'] = ndata3
                data['label'] = labell
                
                temp = file[step].split("/")
                namefile = temp[6].split(" ")
                print(namefile[0])

                datasave = outfile+namefile[0]
                print(datasave)
                np.save(datasave, data)
                break


def main(_):
    run_training()


if __name__ == '__main__':
    tf.app.run()


/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


Number of test videos=1530
loading checkpoint ./models/HMDB_scratch_fix/rgb_scratch_10000_6_64_0.0001_decay_low_split1/i3d_hmdb_model_split1-14999,waiting......
INFO:tensorflow:Restoring parameters from ./models/HMDB_scratch_fix/rgb_scratch_10000_6_64_0.0001_decay_low_split1/i3d_hmdb_model_split1-14999
load complete!
Loading a video clip from /home/ee401_2/Documents/HMDB_new/catch/Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_0...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/catch/Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_0...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/catch/Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_0...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/catch/Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_0...strat_frame = 192
Load

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 256
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 320
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 384
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2...strat_frame = 448
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2...strat_frame = 256
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2...strat_frame = 320
3
(192, 1024)
Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2
./rgb_scratch_data_split1/test64/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_fr_goo_2
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Chanta____THAT_IS_long_blonde_hair_brush_hair_u_cm_np1_le_goo_0...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_ne

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10...strat_frame = 256
2
(160, 1024)
Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10
./rgb_scratch_data_split1/test64/Prelinger_HabitPat1954_brush_hair_u_cm_np1_le_med_10
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_nm_np1_le_goo_11...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_nm_np1_le_goo_11...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Prelinger_HabitPat1954_brush_hair_u_nm_np1_le_goo_11...strat_fram

...

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10...strat_frame = 192
2
(128, 1024)
Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10
./rgb_scratch_data_split1/test64/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med_10
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Bush_Wave_vs__Obama_Wave_and_Which_Commander-in-chief_Salutes_the_Best__You_Decide_wave_u_cm_np1_fr_med

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9...strat_frame = 192
2
(128, 1024)
DONNIE_DARKO_wave_u_nm_np1_fr_goo_9
./rgb_scratch_data_split1/test64/DONNIE_DARKO_wave_u_nm_np1_fr_goo_9
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_le_goo_8...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_le_goo_8...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_le_goo_8...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/DONNIE_DARKO_wave_u_nm_np1_le_goo_8...strat_frame = 192
2
(128, 1024)
DONNIE_DARKO_wave_u_nm_np1_le_goo_8
./rgb_scratch_data_split1/test64/DONNIE_DARKO_wave_u_nm_np1_le_goo_8
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Eurotrip_wave_u_nm_np1_fr_goo_4...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/H

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_0...strat_frame = 192
2
(128, 1024)
TrumanShow_wave_u_nm_np1_fr_med_0
./rgb_scratch_data_split1/test64/TrumanShow_wave_u_nm_np1_fr_med_0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_25...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_25...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_25...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/TrumanShow_wave_u_nm_np1_fr_med_25...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/

SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
